In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [3]:
import versor as vsr
from versor.drawing import *
from motor_estimation import MotorEstimationSolver
# from game import (MotorEstimationSolver, VDMotorEstimationSolver)

# Rotor Estimation

In [4]:
rotor = vsr.Biv(pi/6, 0.0, 0.0).exp()
print(rotor)

Rot: [ 0.87 -0.5 0 0 ]


In [5]:
vectors_a = [vsr.Vec(1,1,0), vsr.Vec(1,0,1), vsr.Vec(0,1,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [6]:
def project_r3(m):
    r = vsr.Vec(0,0,0)
    r[0] = (vsr.CGA(vsr.Pnt(1,0,0,0,0)) * vsr.CGA(m))[0]
    r[1] = (vsr.CGA(vsr.Pnt(0,1,0,0,0)) * vsr.CGA(m))[0]
    r[2] = (vsr.CGA(vsr.Pnt(0,0,1,0,0)) * vsr.CGA(m))[0]
    return r

In [7]:
project_r3(vsr.Vec(1,2,3).null())

Vec: [ 1 2 3 ]

In [8]:
vsr.CGA(vsr.Vec(1,0,0).null()) * vsr.CGA(vsr.Vec(1,2,3).null())

CGA: [ -6.5 0 0 0 0 0 2 3 0 0 -2 -3 6.5 -1 -1.5 6.5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]

In [9]:
initial_rotor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_rotor)
for a, b in zip(vectors_a, vectors_b):
    mes.add_point_correspondences_residual_block(a.null(),b.null())
# mes.set_parameterization('BIVECTOR_GENERATOR')
mes.set_parameterization('POLAR_DECOMPOSITION_TANGENT')
mes.linear_solver_type = 'DENSE_QR'
# mes.minimizer_type = 'LINE_SEARCH'
# mes.line_search_type = 'ARMIJO'
# mes.line_search_direction_type = 'NONLINEAR_CONJUGATE_GRADIENT'
# mes.nonlinear_conjugate_gradient_type = 'HESTENES_STIEFEL'

(estimated_rotor, summary, rotors) = mes.solve()
print(summary['full_report'])
print(estimated_rotor)


Solver Summary (v 1.12.0-eigen-(3.3.1)-lapack-suitesparse-(4.5.3)-cxsparse-(3.1.9)-no_openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                             3                        3
Residual                                    9                        9

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1
Linear solver ordering              AUTOMATIC        

In [10]:
n_points = 100000
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.3, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(rotor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [11]:
initial_rotor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_rotor)
for a, b in zip(points_a, points_b):
    mes.add_point_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
# mes.set_parameterization('POLAR_DECOMPOSITION_TANGENT')
# mes.set_parameterization('POLAR_DECOMPOSITION')
mes.linear_solver_type = 'DENSE_QR'
# mes.minimizer_type = 'LINE_SEARCH'
# mes.line_search_type = 'ARMIJO'
# mes.line_search_direction_type = 'NONLINEAR_CONJUGATE_GRADIENT'
# mes.nonlinear_conjugate_gradient_type = 'HESTENES_STIEFEL'

(estimated_rotor, summary, rotors) = mes.solve()
print(summary['full_report'])
print(estimated_rotor)


Solver Summary (v 1.12.0-eigen-(3.3.1)-lapack-suitesparse-(4.5.3)-cxsparse-(3.1.9)-no_openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                        100000                   100000
Residual                               300000                   300000

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1
Linear solver ordering              AUTOMATIC        

In [12]:
mes.line_search_type?

In [13]:
scene = Scene(children=
              [vector_mesh(a,arrow=True,color='hotpink') for a in vectors_a] +\
              [vector_mesh(b,arrow=True,color='lightskyblue') for b in vectors_b] +\
              [frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')

display(renderer)

# Circles

In [14]:
motor = vsr.Vec(0.5,1,1.5).trs() * vsr.Biv(pi/6, 0.0, 0.0).exp()
p1s = [vsr.Vec(1,0,0).null(), vsr.Vec(0,1,0).null(), vsr.Vec(0,0,1).null()] 
p2s = [p1i.spin(motor) for p1i in p1s] 
p3s = [(p2i - p1i).vec().null() for p1i, p2i in zip(p1s,p2s)]
c1 = vsr.Cir(*p1s)
c2 = vsr.Cir(*p2s)
c3 = vsr.Cir(*p3s)
pp1 = c1.dual().split() 
(pp1[1] - pp1[0]).vec().norm() == 2.0 * c1.radius()

True

In [15]:
c1.radius()

0.816496580927726

In [16]:
print(p3s)

[Pnt: [ 2.22e-16 1.866 1.5 1 2.866 ], Pnt: [ -0.366 0.5 1.5 1 1.317 ], Pnt: [ 0.5 1 1.5 1 1.75 ]]


In [17]:
class Colors:
    RED = 'red'
    GREEN = 'green'
    BLUE = 'blue'
    HOTPINK = 'hotpink'
    LIGHTSKYBLUE = 'lightskyblue'
    AQUAMARINE = 'aquamarine'

In [18]:
Colors.LIGHTSKYBLUE

'lightskyblue'

In [19]:
scene = Scene(children=
              [point_mesh(p1i,color=Colors.HOTPINK) for p1i in p1s] +\
#               [point_mesh(p2i,color='lightskyblue') for p2i in p2s] +\
              
              [circle_mesh(c1, color='hotpink')] +\
              [point_mesh(p, color='red') for p in  pp1] +\
#               [circle_mesh(c2, color='lightskyblue')] +\
#               [circle_mesh(c3, color='turquoise')] +\
#               [circle_mesh(c2 - c1, color='red')] +\
              [line_mesh(vsr.Dll(*pp1), color=Colors.GREEN)] +\

              [frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [20]:
pp = c1.dual()
pp.__print_debug_info_console()

# Tangents

In [21]:
t1 = vsr.Tnv(vsr.Vec(1,2,3).null(), vsr.Vec(-1,1,1).unit())
t2 = vsr.Tnv(vsr.Vec(1,3,3).null(), vsr.Vec(-1,2,1).unit())
t3 = t2.comm(t1).dual()
print(t1.radius())
print(t2.radius())
print(t3.radius())

AttributeError: module 'versor' has no attribute 'Tnv'

In [22]:
def tangent_vector_mesh(tnv, color='gray'):
    tangent = point_mesh(tnv.pnt(), color=color)
    tangent.children = [vector_mesh(tnv.dir().vec(), arrow=True, color=color)]
    return tangent
    

In [23]:
scene = Scene(children=
              [tangent_vector_mesh(t1, color=Colors.LIGHTSKYBLUE),
               tangent_vector_mesh(t2, color=Colors.LIGHTSKYBLUE),
               circle_mesh(t3, color=Colors.HOTPINK),
               frame_mesh(),
               AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

NameError: name 't1' is not defined

In [24]:
p = vsr.Par(vsr.Vec(1,2,3).null(), vsr.Vec(-1,1,1).null())
p

Par: [ 3 4 -1 2 1 2 8.5 -4 -2.5 -5.5 ]

In [25]:
a = vsr.Vec(1,0,0).null()
b = vsr.Vec(-1,0,0).null()
l = vsr.Dll(a,b)
l.dir()

Drv: [ -2 0 0 ]

In [26]:
vsr.Par(a,b).pnt()

Pnt: [ 0 0 0 1 0 ]

In [27]:
scene = Scene(children=
              [point_mesh(a), point_mesh(b)] +\
              [circle_mesh(t.dual(), color='hotpink')] +\

              [line_mesh(vsr.Dll(a,b), color=Colors.GREEN)] +\
              [frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

NameError: name 't' is not defined